In [1]:
from pathlib import Path
import glob
import os
import sys
import csv
import nltk
from collections import Counter
from nltk.tokenize import word_tokenize,wordpunct_tokenize
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures, TrigramCollocationFinder, TrigramAssocMeasures

import locale
# Set the locale to your desired setting (e.g., 'fr_FR.UTF-8' for French)
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [2]:
sys.path.append("/home/lucas-jerusalimiec/Documents/OCR Text/Notebooks")
from tokenizer_func  import (wordcleaner, write_words_to_file, dictionary_to_file, convert_tuple_bigrams,
convert_tuple_trigrams)

from extra_token_func import print_first_n_items, remove_keys_from_nested_dict

from additional_token_func import convert_strings_to_counts

from dict_write import write_dict_to_files_with_suffix

In [3]:
text_loc = Path("./final")
text_files = sorted(glob.glob(f"{text_loc}/*.txt"), key=locale.strxfrm)
output_folder = './tokenized/'
tokenized_folder = Path(output_folder)
tokenized_folder.mkdir(exist_ok=True)

output_unigram = f'{output_folder}unigram_counts'
unigram_folder = Path(output_unigram)
unigram_folder.mkdir(exist_ok=True)

output_bigram = f'{output_folder}bigram_counts'
bigram_folder = Path(output_bigram)
bigram_folder.mkdir(exist_ok=True)

output_trigram = f'{output_folder}trigram_counts'
trigram_folder = Path(output_trigram)
trigram_folder.mkdir(exist_ok=True)

output_collocation = f'{output_folder}collocation_counts'
collocation_folder = Path(output_collocation)
collocation_folder.mkdir(exist_ok=True)

output_trigram_collocation = f'{output_folder}trigram_collocation_counts'
trigram_collocation_folder = Path(output_trigram_collocation)
trigram_collocation_folder.mkdir(exist_ok=True)

output_underscore = f'{output_folder}underscore_bigrams'
underscore_folder = Path(output_underscore)
underscore_folder.mkdir(exist_ok=True)

output_trigram_underscore = f'{output_folder}underscore_trigrams'
trigram_underscore_folder = Path(output_trigram_underscore)
trigram_underscore_folder.mkdir(exist_ok=True)

print("Text files in the spellchecked directory:", text_files)

Text files in the spellchecked directory: ['final/République I.1_corrected.txt', 'final/République I.2_corrected.txt', 'final/République I.3_corrected.txt', 'final/République I.4_corrected.txt', 'final/République I.5_corrected.txt', 'final/République I.6_corrected.txt', 'final/République I.7_corrected.txt', 'final/République I.8_corrected.txt', 'final/République I.910_corrected.txt', 'final/République I.911_corrected.txt', 'final/République I.9_corrected.txt', 'final/République II.1_corrected.txt', 'final/République II.2_corrected.txt', 'final/République II.3_corrected.txt', 'final/République II.4_corrected.txt', 'final/République II.5_corrected.txt', 'final/République II.6_corrected.txt', 'final/République II.7_corrected.txt', 'final/République III.1_corrected.txt', 'final/République III.2_corrected.txt', 'final/République III.3_corrected.txt', 'final/République III.4_corrected.txt', 'final/République III.5_corrected.txt', 'final/République III.6_corrected.txt', 'final/République III.

In [4]:
# Open stopwords CSV file and list the contents
with open('./stop_words.csv', 'r') as f:
    stopwords = f.read().strip().split(",")
stopwords[-10:]

['votr', 'voz', 'w', 'x', 'xx', 'y', 'ya', 'ycel', 'ure', 'z']

In [5]:
tokenized_texts = {}
for txt in text_files:
    with open(txt, 'r') as f:
        content = f.read()
        file_name = txt.split('\\')[-1]
        #key = file_name.split('.')[0]
        key = os.path.splitext(os.path.basename(file_name))[0]
        tokenized_texts[key] = content
print("Raw texts:", list(tokenized_texts.keys()))       

Raw texts: ['République I.1_corrected', 'République I.2_corrected', 'République I.3_corrected', 'République I.4_corrected', 'République I.5_corrected', 'République I.6_corrected', 'République I.7_corrected', 'République I.8_corrected', 'République I.910_corrected', 'République I.911_corrected', 'République I.9_corrected', 'République II.1_corrected', 'République II.2_corrected', 'République II.3_corrected', 'République II.4_corrected', 'République II.5_corrected', 'République II.6_corrected', 'République II.7_corrected', 'République III.1_corrected', 'République III.2_corrected', 'République III.3_corrected', 'République III.4_corrected', 'République III.5_corrected', 'République III.6_corrected', 'République III.7_corrected', 'République IV.1_corrected', 'République IV.2_corrected', 'République IV.3_corrected', 'République IV.4_corrected', 'République IV.5_corrected', 'République IV.6_corrected', 'République IV.7_corrected', 'République preface_corrected', 'République V.1_corrected', 

In [6]:
unigrams = {}

for key, value in tokenized_texts.items():
    unigram_list = wordpunct_tokenize(value)
    cleanwords = [wordcleaner(w) for w in unigram_list]
    unigrams[key] = cleanwords

for key, value in unigrams.items():
    filename = f"./tokenized/{key}.txt"
    write_words_to_file(value, filename, words_per_line=20)
    print(f"Saved content for '{key}' to {filename}")

Saved content for 'République I.1_corrected' to ./tokenized/République I.1_corrected.txt
Saved content for 'République I.2_corrected' to ./tokenized/République I.2_corrected.txt
Saved content for 'République I.3_corrected' to ./tokenized/République I.3_corrected.txt
Saved content for 'République I.4_corrected' to ./tokenized/République I.4_corrected.txt
Saved content for 'République I.5_corrected' to ./tokenized/République I.5_corrected.txt
Saved content for 'République I.6_corrected' to ./tokenized/République I.6_corrected.txt
Saved content for 'République I.7_corrected' to ./tokenized/République I.7_corrected.txt
Saved content for 'République I.8_corrected' to ./tokenized/République I.8_corrected.txt
Saved content for 'République I.910_corrected' to ./tokenized/République I.910_corrected.txt
Saved content for 'République I.911_corrected' to ./tokenized/République I.911_corrected.txt
Saved content for 'République I.9_corrected' to ./tokenized/République I.9_corrected.txt
Saved content

In [7]:
print("Unigram texts:")
for key in unigrams:
    print(key)

Unigram texts:
République I.1_corrected
République I.2_corrected
République I.3_corrected
République I.4_corrected
République I.5_corrected
République I.6_corrected
République I.7_corrected
République I.8_corrected
République I.910_corrected
République I.911_corrected
République I.9_corrected
République II.1_corrected
République II.2_corrected
République II.3_corrected
République II.4_corrected
République II.5_corrected
République II.6_corrected
République II.7_corrected
République III.1_corrected
République III.2_corrected
République III.3_corrected
République III.4_corrected
République III.5_corrected
République III.6_corrected
République III.7_corrected
République IV.1_corrected
République IV.2_corrected
République IV.3_corrected
République IV.4_corrected
République IV.5_corrected
République IV.6_corrected
République IV.7_corrected
République preface_corrected
République V.1_corrected
République V.2_corrected
République V.3_corrected
République V.4_corrected
République V.5_corrected

In [8]:
# Count up the tokens using a Counter() object
unigram_counts = {}
for key, value in unigrams.items():
    unigram_counts_dict = Counter(value)
    unigram_counts[key] = unigram_counts_dict

print("Unigram Counts:")
for key in unigram_counts:
    print(key)

print_first_n_items(unigram_counts, 25)

Unigram Counts:
République I.1_corrected
République I.2_corrected
République I.3_corrected
République I.4_corrected
République I.5_corrected
République I.6_corrected
République I.7_corrected
République I.8_corrected
République I.910_corrected
République I.911_corrected
République I.9_corrected
République II.1_corrected
République II.2_corrected
République II.3_corrected
République II.4_corrected
République II.5_corrected
République II.6_corrected
République II.7_corrected
République III.1_corrected
République III.2_corrected
République III.3_corrected
République III.4_corrected
République III.5_corrected
République III.6_corrected
République III.7_corrected
République IV.1_corrected
République IV.2_corrected
République IV.3_corrected
République IV.4_corrected
République IV.5_corrected
République IV.6_corrected
République IV.7_corrected
République preface_corrected
République V.1_corrected
République V.2_corrected
République V.3_corrected
République V.4_corrected
République V.5_correcte

In [9]:
# Remove specified keys from the dictionary
stripped_unigrams = remove_keys_from_nested_dict(unigram_counts, stopwords)

print_first_n_items(stripped_unigrams, 30)

First 30 items in République I.1_corrected:
: 858
bien: 39
republique: 22
fin: 12
choses: 12
contemplation: 12
felicité: 12
moins: 10
gift: 10
homme: 10
ame: 10
but: 8
souverain: 8
actions: 8
droit: 7
vertu: 7
hommes: 7
corps: 7
cela: 6
vie: 6
vraye: 6
vertus: 6
morales: 6
principale: 5
gouvernement: 5
premier: 5
tousjours: 5
republiques: 5
doit: 5
loix: 5

First 30 items in République I.2_corrected:
: 1053
republique: 31
famille: 18
familles: 13
bien: 12
enfans: 9
droit: 8
plusieurs: 8
propre: 8
cité: 8
particulier: 8
puissance: 7
fans: 7
public: 7
commun: 7
traitez: 7
entre: 6
chef: 6
personnes: 6
femmes: 6
ville: 6
chefs: 6
loix: 6
chose: 6
coustumes: 6
biens: 6
gouvernement: 5
menages: 5
peuple: 5
roy: 5

First 30 items in République I.3_corrected:
: 1511
mari: 45
femme: 44
puissance: 30
femmes: 18
loy: 16
bien: 16
cause: 13
pere: 11
fans: 11
maris: 11
encores: 9
faire: 7
coustume: 7
droit: 7
mariage: 7
aufsi: 7
lafemme: 7
peine: 7
mort: 7
romains: 7
loix: 6
famille: 6
toute: 6
enf

In [10]:
dictionary_to_file(stripped_unigrams, unigram_folder, 'unigram_counts')

Saved République I.1_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République I.2_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République I.3_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République I.4_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République I.5_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République I.6_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République I.7_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République I.8_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République I.910_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République I.911_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République I.9_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République II.1_corrected_unigram_counts.csv in tokenized/unigram_counts
Saved République II.2_corrected_unigram_counts.csv in token

In [11]:
bigrams = {}

for key, value in unigrams.items():
    unigram_list = [word for word in value if word.lower() not in stopwords]
    #unigram_list = [word for word in value]
    bigrams_list = list(nltk.bigrams(unigram_list))
    bigrams[key] = bigrams_list

print("Bigrams:")
for key in bigrams:
    print(key)

Bigrams:
République I.1_corrected
République I.2_corrected
République I.3_corrected
République I.4_corrected
République I.5_corrected
République I.6_corrected
République I.7_corrected
République I.8_corrected
République I.910_corrected
République I.911_corrected
République I.9_corrected
République II.1_corrected
République II.2_corrected
République II.3_corrected
République II.4_corrected
République II.5_corrected
République II.6_corrected
République II.7_corrected
République III.1_corrected
République III.2_corrected
République III.3_corrected
République III.4_corrected
République III.5_corrected
République III.6_corrected
République III.7_corrected
République IV.1_corrected
République IV.2_corrected
République IV.3_corrected
République IV.4_corrected
République IV.5_corrected
République IV.6_corrected
République IV.7_corrected
République preface_corrected
République V.1_corrected
République V.2_corrected
République V.3_corrected
République V.4_corrected
République V.5_corrected
Répub

In [12]:
bigram_counts = {}

for key, value in bigrams.items():
    string_bigrams = convert_tuple_bigrams(value)
    bigramCount = convert_strings_to_counts(string_bigrams)
    bigram_counts[key] = bigramCount

print("Bigram Counts:")
for key in bigram_counts:
    print(key)

print_first_n_items(bigram_counts, 30)

Bigram Counts:
République I.1_corrected
République I.2_corrected
République I.3_corrected
République I.4_corrected
République I.5_corrected
République I.6_corrected
République I.7_corrected
République I.8_corrected
République I.910_corrected
République I.911_corrected
République I.9_corrected
République II.1_corrected
République II.2_corrected
République II.3_corrected
République II.4_corrected
République II.5_corrected
République II.6_corrected
République II.7_corrected
République III.1_corrected
République III.2_corrected
République III.3_corrected
République III.4_corrected
République III.5_corrected
République III.6_corrected
République III.7_corrected
République IV.1_corrected
République IV.2_corrected
République IV.3_corrected
République IV.4_corrected
République IV.5_corrected
République IV.6_corrected
République IV.7_corrected
République preface_corrected
République V.1_corrected
République V.2_corrected
République V.3_corrected
République V.4_corrected
République V.5_corrected

In [13]:
dictionary_to_file(bigram_counts, bigram_folder, 'bigram_counts')

Saved République I.1_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République I.2_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République I.3_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République I.4_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République I.5_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République I.6_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République I.7_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République I.8_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République I.910_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République I.911_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République I.9_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République II.1_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved République II.2_corrected_bigram_counts.csv in tokenized/bigram_counts
Saved 

In [14]:
trigrams = {}

for key, value in unigrams.items():
    unigram_list = [word for word in value if word.lower() not in stopwords]
    #unigram_list = [word for word in value]
    trigrams_list = list(nltk.trigrams(unigram_list))
    trigrams[key] = trigrams_list

print("Trigrams:")
for key in bigrams:
    print(key)

Trigrams:
République I.1_corrected
République I.2_corrected
République I.3_corrected
République I.4_corrected
République I.5_corrected
République I.6_corrected
République I.7_corrected
République I.8_corrected
République I.910_corrected
République I.911_corrected
République I.9_corrected
République II.1_corrected
République II.2_corrected
République II.3_corrected
République II.4_corrected
République II.5_corrected
République II.6_corrected
République II.7_corrected
République III.1_corrected
République III.2_corrected
République III.3_corrected
République III.4_corrected
République III.5_corrected
République III.6_corrected
République III.7_corrected
République IV.1_corrected
République IV.2_corrected
République IV.3_corrected
République IV.4_corrected
République IV.5_corrected
République IV.6_corrected
République IV.7_corrected
République preface_corrected
République V.1_corrected
République V.2_corrected
République V.3_corrected
République V.4_corrected
République V.5_corrected
Répu

In [15]:
trigram_counts = {}

for key, value in trigrams.items():
    string_trigrams = convert_tuple_trigrams(value)
    trigramCount = convert_strings_to_counts(string_trigrams)
    trigram_counts[key] = trigramCount

print("Trigram Counts:")
for key in trigram_counts:
    print(key)
    
print_first_n_items(trigram_counts, 30)

Trigram Counts:
République I.1_corrected
République I.2_corrected
République I.3_corrected
République I.4_corrected
République I.5_corrected
République I.6_corrected
République I.7_corrected
République I.8_corrected
République I.910_corrected
République I.911_corrected
République I.9_corrected
République II.1_corrected
République II.2_corrected
République II.3_corrected
République II.4_corrected
République II.5_corrected
République II.6_corrected
République II.7_corrected
République III.1_corrected
République III.2_corrected
République III.3_corrected
République III.4_corrected
République III.5_corrected
République III.6_corrected
République III.7_corrected
République IV.1_corrected
République IV.2_corrected
République IV.3_corrected
République IV.4_corrected
République IV.5_corrected
République IV.6_corrected
République IV.7_corrected
République preface_corrected
République V.1_corrected
République V.2_corrected
République V.3_corrected
République V.4_corrected
République V.5_correcte

In [16]:
dictionary_to_file(trigram_counts, trigram_folder, 'trigram_counts')

Saved République I.1_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République I.2_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République I.3_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République I.4_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République I.5_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République I.6_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République I.7_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République I.8_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République I.910_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République I.911_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République I.9_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République II.1_corrected_trigram_counts.csv in tokenized/trigram_counts
Saved République II.2_corrected_trigram_counts.csv in token

In [17]:
colloc_dict = {}
colloc_counts = {}

for key, value in unigrams.items():
    unigram_list = [word for word in value if word.lower() not in stopwords]
    bigram_finder = BigramCollocationFinder.from_words(unigram_list)
    bigram_finder.apply_freq_filter(3)  # Make sure all collocations have occurred at least 5 times
    collocations = bigram_finder.nbest(BigramAssocMeasures.pmi, 500)
    filtered_collocations = [bigram for bigram in collocations if '' not in bigram]
    colloc_dict[key] = filtered_collocations
    
    # Initialize Counter for colloc_counts
    bigram_count_dict = Counter()

    # Count the occurrences of each bigram in the text
    bigram_finder = BigramCollocationFinder.from_words(unigram_list)
    bigram_freqs = bigram_finder.ngram_fd.items()
    
    # Filter bigram counts based on collocations
    for bigram, count in bigram_freqs:
        if bigram in filtered_collocations:
            bigram_count_dict[bigram] = count

    colloc_counts[key] = bigram_count_dict

print("Collocations:")
for key, value in colloc_dict.items():
    print(key)
    # for w1, w2 in value:
    #     print(' ', w1, w2)

print("Collocation Counts:")
for key in colloc_counts:
    print(key)
    # Print first n items, assuming print_first_n_items function is defined elsewhere
    for item, count in colloc_counts[key].most_common(30):
        bigram = " ".join(item)
        print(f"{bigram} {count}")
    print()

dictionary_to_file(colloc_counts, collocation_folder, 'collocation_counts')

Collocations:
République I.1_corrected
République I.2_corrected
République I.3_corrected
République I.4_corrected
République I.5_corrected
République I.6_corrected
République I.7_corrected
République I.8_corrected
République I.910_corrected
République I.911_corrected
République I.9_corrected
République II.1_corrected
République II.2_corrected
République II.3_corrected
République II.4_corrected
République II.5_corrected
République II.6_corrected
République II.7_corrected
République III.1_corrected
République III.2_corrected
République III.3_corrected
République III.4_corrected
République III.5_corrected
République III.6_corrected
République III.7_corrected
République IV.1_corrected
République IV.2_corrected
République IV.3_corrected
République IV.4_corrected
République IV.5_corrected
République IV.6_corrected
République IV.7_corrected
République preface_corrected
République V.1_corrected
République V.2_corrected
République V.3_corrected
République V.4_corrected
République V.5_corrected


In [18]:
trigram_colloc_dict = {}
trigram_colloc_counts = {}

for key, value in unigrams.items():
    unigram_list = [word for word in value if word.lower() not in stopwords]
    trigram_finder = TrigramCollocationFinder.from_words(unigram_list)
    trigram_finder.apply_freq_filter(3)  # Ensure all collocations have occurred at least 5 times
    collocations = trigram_finder.nbest(TrigramAssocMeasures.pmi, 500)
    filtered_collocations = [trigram for trigram in collocations if '' not in trigram]
    trigram_colloc_dict[key] = filtered_collocations
    
    # Initialize Counter for trigram_colloc_counts
    trigram_count_dict = Counter()

    # Count the occurrences of each trigram in the text
    trigram_freqs = trigram_finder.ngram_fd.items()
    
    # Filter trigram counts based on collocations
    for trigram, count in trigram_freqs:
        if trigram in filtered_collocations:
            trigram_count_dict[trigram] = count

    trigram_colloc_counts[key] = trigram_count_dict

print("Trigram Collocations:")
for key, value in trigram_colloc_dict.items():
    print(key)
    #for w1, w2, w3 in value:
    #    print(' ', w1, w2, w3)

print("Trigram Collocation Counts:")
for key in trigram_colloc_counts:
    print(key)
    # Print first n items, assuming print_first_n_items function is defined elsewhere
    for item, count in trigram_colloc_counts[key].most_common(30):
        trigram = " ".join(item)
        print(f"{trigram} {count}")
    print()

dictionary_to_file(trigram_colloc_counts, trigram_collocation_folder, 'trigram_collocation_counts')

Trigram Collocations:
République I.1_corrected
République I.2_corrected
République I.3_corrected
République I.4_corrected
République I.5_corrected
République I.6_corrected
République I.7_corrected
République I.8_corrected
République I.910_corrected
République I.911_corrected
République I.9_corrected
République II.1_corrected
République II.2_corrected
République II.3_corrected
République II.4_corrected
République II.5_corrected
République II.6_corrected
République II.7_corrected
République III.1_corrected
République III.2_corrected
République III.3_corrected
République III.4_corrected
République III.5_corrected
République III.6_corrected
République III.7_corrected
République IV.1_corrected
République IV.2_corrected
République IV.3_corrected
République IV.4_corrected
République IV.5_corrected
République IV.6_corrected
République IV.7_corrected
République preface_corrected
République V.1_corrected
République V.2_corrected
République V.3_corrected
République V.4_corrected
République V.5_co

In [19]:
underscore_dict = {}
for key, value in unigrams.items():

    tokenized_words = unigrams.get(key)
    collocations = colloc_dict.get(key)
    
    colloc_words = []
    
    # Iterate through the words making new versions combining collocations
    i = 0
    while i < len(tokenized_words) - 1:
        # If we find a collocation, add and advance by two words
        if (tokenized_words[i], tokenized_words[i + 1]) in collocations:
            colloc_words.append('_'.join((tokenized_words[i], tokenized_words[i + 1])))
            i += 2
        # Otherwise, advance by one word
        else:
            colloc_words.append(tokenized_words[i])
            i += 1

    # Add the last word (if any)
    if i == len(tokenized_words) - 1:
        colloc_words.append(tokenized_words[i])
    underscore_dict[key] = colloc_words

print("Underscore Dictionary:")
for key in underscore_dict:
    print(key)

write_dict_to_files_with_suffix(underscore_dict, underscore_folder, 'underscore_bigrams')

Underscore Dictionary:
République I.1_corrected
République I.2_corrected
République I.3_corrected
République I.4_corrected
République I.5_corrected
République I.6_corrected
République I.7_corrected
République I.8_corrected
République I.910_corrected
République I.911_corrected
République I.9_corrected
République II.1_corrected
République II.2_corrected
République II.3_corrected
République II.4_corrected
République II.5_corrected
République II.6_corrected
République II.7_corrected
République III.1_corrected
République III.2_corrected
République III.3_corrected
République III.4_corrected
République III.5_corrected
République III.6_corrected
République III.7_corrected
République IV.1_corrected
République IV.2_corrected
République IV.3_corrected
République IV.4_corrected
République IV.5_corrected
République IV.6_corrected
République IV.7_corrected
République preface_corrected
République V.1_corrected
République V.2_corrected
République V.3_corrected
République V.4_corrected
République V.5_c

In [20]:
trigram_underscore_dict = {}

for key, tokenized_words in unigrams.items():
    collocations = trigram_colloc_dict.get(key, [])
    colloc_words = []
    i = 0
    while i < len(tokenized_words) - 2:
        # If we find a trigram collocation, add and advance by three words
        trigram = (tokenized_words[i], tokenized_words[i + 1], tokenized_words[i + 2])
        if trigram in collocations:
            colloc_words.append('_'.join(trigram))
            i += 3
        else:
            colloc_words.append(tokenized_words[i])
            i += 1
    # Add the last words (if any)
    while i < len(tokenized_words):
        colloc_words.append(tokenized_words[i])
        i += 1
    trigram_underscore_dict[key] = colloc_words

print("Trigram underscore Dictionary:")
for key in trigram_underscore_dict:
    print(key)

write_dict_to_files_with_suffix(trigram_underscore_dict, trigram_underscore_folder, 'underscore_trigrams')

Trigram underscore Dictionary:
République I.1_corrected
République I.2_corrected
République I.3_corrected
République I.4_corrected
République I.5_corrected
République I.6_corrected
République I.7_corrected
République I.8_corrected
République I.910_corrected
République I.911_corrected
République I.9_corrected
République II.1_corrected
République II.2_corrected
République II.3_corrected
République II.4_corrected
République II.5_corrected
République II.6_corrected
République II.7_corrected
République III.1_corrected
République III.2_corrected
République III.3_corrected
République III.4_corrected
République III.5_corrected
République III.6_corrected
République III.7_corrected
République IV.1_corrected
République IV.2_corrected
République IV.3_corrected
République IV.4_corrected
République IV.5_corrected
République IV.6_corrected
République IV.7_corrected
République preface_corrected
République V.1_corrected
République V.2_corrected
République V.3_corrected
République V.4_corrected
Républiq